<a href="https://colab.research.google.com/github/javatcoding1/MRI-Prediction/blob/main/site/en/gemini-api/docs/vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q -U google-generativeai

In [4]:
import google.generativeai as genai
from IPython.display import Markdown

In [5]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [10]:
import httpx
import base64
import mimetypes

# Path to the image
image_path = "/content/images.jpeg"  # Replace with any image

# Detect MIME type dynamically
mime_type = mimetypes.guess_type(image_path)[0] or "image/jpeg"  # Default to JPEG if unknown

# Read image file as binary data
with open(image_path, "rb") as image_file:
    image_data = image_file.read()

# Choose a Gemini model
model = genai.GenerativeModel(model_name="gemini-1.5-pro")

# System prompt for structured output
prompt = """You are analyzing a medical scan image. Provide the following structured output:

- **Scan Type:** (e.g., MRI, CT Scan, X-ray)
- **Organ:** (e.g., Brain, Lung, Heart,Breast)
- **Tumor Type:** (If detected, specify type)
- **Tumor Subclass:** (If applicable)
- **Detailed Description:** (Size, shape, and location if possible)
- **Possible Causes:** (List genetic factors, environmental exposure, lifestyle)
- **Clinical Insights:** (Any relevant medical observations)

Ensure the output is structured in plain text without any extra information or unrelated content."""

# Generate response
response = model.generate_content(
    [
        {
            "mime_type": mime_type,
            "data": base64.b64encode(image_data).decode("utf-8"),
        },
        prompt,
    ]
)

# Extract response text
response_text = response.text

# Print the structured response
print(response_text)

# Extract specific values (optional)
lines = response_text.split("\n")
scan_type = lines[0].replace("Scan Type:", "").strip() if len(lines) > 0 else "Unknown"
organ_type = lines[1].replace("Organ:", "").strip() if len(lines) > 1 else "Unknown"

# Print extracted values (for further use)
print("Scan Type:", scan_type)
print("Organ Type:", organ_type)


- **Scan Type:** CT Scan
- **Organ:** Lung
- **Tumor Type:** Cannot be determined from image alone.  Requires biopsy and pathology. Possibly a lung mass.
- **Tumor Subclass:** Cannot be determined from image alone.
- **Detailed Description:** A large mass is present in the anterior mediastinum, possibly arising from or invading the anterior portion of the right lung. Precise size and shape require measurements from the scan data.  The density of the mass cannot be fully characterized from this single image.
- **Possible Causes:** Cannot be determined from image alone. Lung cancer can be caused by smoking, exposure to radon or asbestos, family history of lung cancer, air pollution, and other factors.  Mediastinal masses can arise from lymphoma, thymoma, germ cell tumors and other entities.
- **Clinical Insights:** Further investigation is absolutely required including biopsy/tissue sampling to confirm the diagnosis, assess malignancy, and guide treatment.  Additional imaging, such as PE

### Multiple images

To prompt with multiple images in Base64 encoded format, you can do the
following:

In [8]:
organ_type

'- **** Lung'

In [12]:
if organ_type == "- **** Brain":
  import tensorflow as tf
  import numpy as np
  from tensorflow.keras.models import load_model
  from tensorflow.keras.preprocessing import image

  # Load the trained model
  model = load_model("brain_model.h5")

  # Model summary (optional, to verify)
  model.summary()

  # Function to preprocess an input image
  def preprocess_image(img_path):
      img_size = (299, 299)  # Match model's input shape
      img = image.load_img(img_path, target_size=img_size)
      img_array = image.img_to_array(img)  # Convert to numpy array
      img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
      img_array = tf.keras.applications.xception.preprocess_input(img_array)  # Apply Xception preprocessing
      return img_array

  # Load and preprocess an MRI image
  img_path = "/content/men.jpeg"  # Replace with your image path
  processed_img = preprocess_image(img_path)

  # Perform prediction
  predictions = model.predict(processed_img)

  # Decode predictions (assuming 4 classes)
  class_labels = ["Class 1", "Class 2", "Class 3", "Class 4"]  # Replace with actual class names
  predicted_class = class_labels[np.argmax(predictions)]

  # Print results
  print("Predicted Class:", predicted_class)
  print("Prediction Probabilities:", predictions)
elif organ_type=="- **** Lung":
  import tensorflow as tf
  import numpy as np
  from tensorflow.keras.models import load_model
  from tensorflow.keras.preprocessing import image

  model = load_model("/content/lung_tumor.h5")  # Replace with your actual .h5 file path
  IMG_WIDTH = 224  # Update this based on your model training
  IMG_HEIGHT = 224

  def preprocess_image(img_path):
      img = image.load_img(img_path, target_size=(IMG_WIDTH, IMG_HEIGHT))
      img_array = image.img_to_array(img)
      img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
      img_array = img_array / 255.0  # Normalize (if your model was trained with normalization)
      return img_array
  img_path = "/content/images.jpeg"  # Replace with the path of the image you want to classify
  img_array = preprocess_image(img_path)

  prediction = model.predict(img_array)
  predicted_class = np.argmax(prediction, axis=1)  # Get class with highest probability

  print(f"Predicted Class: {predicted_class}")
  class_labels = ["Benign", "Malignant", "Normal"]  # Adjust based on your dataset
  print(f"Predicted Class: {class_labels[predicted_class[0]]}")




else:
  import tensorflow as tf
  from tensorflow.keras.models import load_model
  from tensorflow.keras.preprocessing import image
  import numpy as np

  # Load the trained model
  model = load_model("/content/breast_tumor.h5")  # Ensure the correct file path

  # Function to preprocess input image
  def preprocess_image(img_path):
      img = image.load_img(img_path, target_size=(244, 244))  # Resize to match model input
      img_array = image.img_to_array(img)
      img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
      img_array = img_array / 255.0  # Normalize pixel values
      return img_array

  # Path to the test image
  img_path = "/content/SOB_M_DC-14-2523-400-005.png"  # Change this to your actual image path

  # Preprocess and predict
  img_array = preprocess_image(img_path)
  prediction = model.predict(img_array)

  # Interpret the results
  class_labels = ["Benign", "Malignant"]  # Adjust labels if different
  predicted_class = np.argmax(prediction)  # Get class index

  print(f"Predicted Class: {class_labels[predicted_class]}")
  print(f"Confidence Scores: {prediction}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 550ms/step
Predicted Class: [1]
Predicted Class: Malignant


In [2]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

model = load_model("/content/lung_tumor.h5")  # Replace with your actual .h5 file path
IMG_WIDTH = 224  # Update this based on your model training
IMG_HEIGHT = 224

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(IMG_WIDTH, IMG_HEIGHT))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize (if your model was trained with normalization)
    return img_array
img_path = "/content/images.jpeg"  # Replace with the path of the image you want to classify
img_array = preprocess_image(img_path)

prediction = model.predict(img_array)
predicted_class = np.argmax(prediction, axis=1)  # Get class with highest probability

print(f"Predicted Class: {predicted_class}")
class_labels = ["Benign", "Malignant", "Normal"]  # Adjust based on your dataset
print(f"Predicted Class: {class_labels[predicted_class[0]]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 742ms/step
Predicted Class: [1]
Predicted Class: Malignant
